In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from keras import datasets

sys.path.append(os.getcwd() + "/../")

from bfcnn import BFCNN

In [2]:
# ==============================================================================

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.compat.v1.disable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# ==============================================================================

MIN_STD = 0.1
MAX_STD = 10
EPOCHS = 150
STEP_SIZE = 30
LR_DECAY = 0.5
BATCH_SIZE = 32
INITIAL_EPOCH = 0
CLIP_NORMAL = 1.0
LEARNING_RATE = 0.01
INPUT_SHAPE = (28, 28, 1)
PRINT_EVERY_N_BATCHES = 200


In [3]:
# build model
model = BFCNN(input_dims=INPUT_SHAPE)

In [ ]:
(x_train, y_train), _ = datasets.mnist.load_data()

trained_model = BFCNN.train(model=model.trainable_model, 
                            input_dims=INPUT_SHAPE, 
                            dataset=x_train,
                            epochs=EPOCHS,
                            batch_size=BATCH_SIZE,
                            min_noise_std=MIN_STD,
                            max_noise_std=MAX_STD,
                            print_every_n_batches=PRINT_EVERY_N_BATCHES)

2021-03-23 17:53:34,006 INFO model.py:train:249] begin training | batches per epoch 1875
2021-03-23 17:53:34,007 INFO model.py:train:253] epoch 0
/home/arxwn/repositories/blind_image_denoising/venv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2021-03-23 17:54:17,521 INFO model.py:train:275] [batch:200] loss: 19.96
2021-03-23 17:55:00,946 INFO model.py:train:275] [batch:400] loss: 9.22
2021-03-23 17:55:42,461 INFO model.py:train:275] [batch:600] loss: 5.89
2021-03-23 17:56:24,846 INFO model.py:train:275] [batch:800] loss: 4.00
2021-03-23 17:57:11,062 INFO model.py:train:275] [batch:1000] loss: 2.69
2021-03-23 17:57:57,821 INFO model.py:train:275] [batch:1200] loss: 4.26
2021-03-23 17:58:45,815 INFO model.py:train:275]